In [2]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
import parser.parser as parser
from lxml import etree
import re
import os
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import Graph, plugin
from rdflib.serializer import Serializer
import rdflib
from rdflib.namespace import DCTERMS

INFO:rdflib:RDFLib Version: 4.2.1


In [3]:
rootpath = '/media/sf_VBox_Shared/CaseLaw/OpenDataUitspraken/'
ecli = 'ECLI:NL:HR:2009:BH2815'
element = parser.retrieve_from_any(ecli, rootpath)

In [4]:
g = Graph()
rdf = list(element.iterdescendants('{*}RDF'))[0]
nsmap = rdf.nsmap
for k, v in nsmap.items():
    g.bind(k, v)
descriptions = list(rdf.iterchildren('{*}Description'))

In [5]:
descriptions

[<Element {http://www.w3.org/1999/02/22-rdf-syntax-ns#}Description at 0x7f8f92fd5fc8>,
 <Element {http://www.w3.org/1999/02/22-rdf-syntax-ns#}Description at 0x7f8f92fdc0c8>]

In [6]:
version_list = parser.get_from_descriptions(descriptions, 'dcterms', 'hasVersion')
for version in version_list:
    value = version.attrib.get('resourceIdentifier', None)
    if value == 'http://psi.rechtspraak.nl/vindplaats':
        print(rdflib.URIRef(value))

http://psi.rechtspraak.nl/vindplaats


In [7]:
for l in parser.get_children(version, 'rdf', 'list'):
    for i in parser.get_children(l, 'rdf', 'li'):
        print(i.text)

Rechtspraak.nl
RvdW 2009, 683
JOR 2009/199 met annotatie van mr. C.W.M. Lieverse onder «JOR» 2004/105
JA 2009/117 met annotatie van W.H. van Boom
NJB 2009, 1206
JONDR 2009, 322
TGMA 2009/17
NJ 2012/182 met annotatie van J.B.M. Vranken
JE 2009, 322


In [8]:
ecli_node = parser.get_identifier(descriptions, ecli)
parser.add_version(descriptions, g, ecli_node)

<Graph identifier=N48b598291b3a4679bbdd3a64f514e039 (<class 'rdflib.graph.Graph'>)>

In [9]:
for s, p, o in g:
    print(p, o)

http://purl.org/dc/terms/hasVersion RvdW 2009, 683
http://purl.org/dc/terms/hasVersion Rechtspraak.nl
http://purl.org/dc/terms/hasVersion JOR 2009/199 met annotatie van mr. C.W.M. Lieverse onder «JOR» 2004/105
http://purl.org/dc/terms/hasVersion JE 2009, 322
http://purl.org/dc/terms/hasVersion JONDR 2009, 322
http://purl.org/dc/terms/hasVersion JA 2009/117 met annotatie van W.H. van Boom
http://purl.org/dc/terms/hasVersion NJB 2009, 1206
http://purl.org/dc/terms/hasVersion NJ 2012/182 met annotatie van J.B.M. Vranken
http://purl.org/dc/terms/hasVersion TGMA 2009/17


In [10]:
reference_list = parser.get_from_descriptions(descriptions, 'dcterms', 'references')
for reference in reference_list:
    uri = reference.attrib.get('bwb:resourceIdentifier', None)
    dtype = reference.attrib.get('rdfs:label', None)
    g.add((ecli_node, DCTERMS.reference, rdflib.URIRef(uri)))
    g.add((rdflib.URIRef(uri), DCTERMS.type, rdflib.Literal(dtype)))

TypeError: argument of type 'NoneType' is not iterable

In [28]:
reference.attrib.get(g.na'bwb'+'resourceIdentifier', None)

{'{http://www.w3.org/2000/01/rdf-schema#}label': 'Wetsverwijzing', '{bwb-dl}resourceIdentifier': '1.0:v:BWB:BWBR0004815&g=2009-06-05'}

In [12]:
g.namespace_manager.normalizeUri('bwb:resourceIdentifier')

'<bwb:resourceIdentifier>'